# Data wrangling with "Azure ML Package for Forecasting"

## Summary
 This notebook demonstrates how to use the primary data structure - `TimeSeriesDataFrame` that comes with `Azure Machine Learning Package for Forecasting`(AMLPF). The `TimeSeriesDataFrame` is at the core of the package and it subclasses the [Pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). In this notebook, we will demonstrate some common data wrangling tasks that involve the `TimeSeriesDataFrame` on the [U. Chicago Dominick's Finer Foods dataset](https://research.chicagobooth.edu/kilts/marketing-databases/dominicks). 
 
 The following data wrangling tasks are shown:
 * Loading datasets
 * Working with Date strings
 * Creating TimeSeriesDataFrame
 * Changing shape of TimeSeriesDataFrame
 * Slicers with TimeSeriesDataFrame
 * Aggregation operations with TimeSeriesDataFrame
 * Merge operations with TimeSeriesDataFrame
 
**Please refer to our [programming guide](https://azuremlftkrelease.blob.core.windows.net/docsdev/html/index.html) for a full list of transfomers and models available in AMLPF and try them out.**
 

In [8]:
import warnings

# Suppress warnings
warnings.filterwarnings("ignore") 

# Import packages
import pandas as pd
import numpy as np
import math
import pkg_resources
from datetime import timedelta
import datetime
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt
from sklearn.linear_model import Lasso

from ftk import TimeSeriesDataFrame
from ftk.data import get_a_year_of_daily_weather_data

## Load Data and Explore

Any [IO tools](https://pandas.pydata.org/pandas-docs/stable/io.html) supported by Pandas can be used to load data. One of the most common ways to do so is the `read_csv` API since many datasets are distributed in that format.
In addittion to wide variety of formats, Pandas provides many ways to handle large datasets. One of the most common ways to handle is by loading them in chunks. Columnar/vector representation of datasets can be compressed to a greater degree that relational frames and are another good option.

In [ ]:
# Read raw data from CSV
csv_path = pkg_resources.resource_filename('ftk', 'data/dominicks_oj/dominicks_oj.csv')
whole_df = pd.read_csv(csv_path, low_memory = False)
whole_df.head()

# Read large datasets in chunks
chunk_size = 500
chunks = []
for chunk in pd.read_csv(csv_path, chunksize=chunk_size):    
    chunks.append(chunk)
whole_df_chunked = pd.concat(chunks, axis=0)
whole_df_chunked.head()

## Working with Date strings

Pandas dataframe offers extensive support to working with time-series and thus Date objects in general. A common
requirement is to convert datetime strings to DateTime objects. Pandas has datetime objects that model them as a single
point in time via the `Timestamp` or a as a time range/period - `Period`. In `TimeSeriesDataFrame` we assume the Date index to be of `Timestamp` type and hence the time axis is of type `DateTimeIndex`. This column is part of the multi-index and stored in the `time_colname` metadata property.

In [ ]:
# The sales are contained in the 'logmove' column. 
# Values are logarithmic, so exponentiate and round them to get quantity sold
def expround(x):
    return math.floor(math.exp(x) + 0.5)
whole_df['quantity'] = whole_df['logmove'].apply(expround)

# Create `TimeStamp` using `to_datetime` passing in a string value 
weekZeroStart = pd.to_datetime('1989-09-07 00:00:00')
print(weekZeroStart)

# Create `TimeStamp` passing in a string value 
weekZeroStartTS = pd.Timestamp('1989-09-07 00:00:00')
print(weekZeroStartTS)

# Create `TimeStamp` with a specific format. The `to_datetime` supports
# formatters.
weekZeroStartFMT = pd.to_datetime('1989-09-07 00:00:00', format='%Y/%m/%d')
print(weekZeroStartFMT)

weekZeroEnd = pd.to_datetime('1989-09-13 23:59:59')
whole_df['WeekFirstDay'] = whole_df['week'].apply(lambda n: weekZeroStart + timedelta(weeks=n))
whole_df['WeekLastDay'] = whole_df['week'].apply(lambda n: weekZeroEnd + timedelta(weeks=n))
whole_df[['store','brand','WeekLastDay', 'quantity']].head()

## Create TimeSeriesDataframe

In this section below, we create a `TimeSeriesDataFrame` from a Pandas dataframe. In the TimeSeriesDataFrame representation, the time axis and grain are part of the data frame index - which is a composite or a MultiIndex in Pandas. This allows easy access to Pandas datetime slicing capabilities.

A MultiIndex is a multi-level or hierarchical representation for Pandas objects. Such an index indicates
the usage of two or more columns to uniquely differentiate or locate records. In database terms, MultiIndex is simply
a [compound](https://en.wikipedia.org/wiki/Compound_key) or composite key as used in database parlance.

You can also glean common information about the `TimeSeriesDataFrame` by using Pandas' APIs.
Some of the common statistics involve finding the shape of the dataframe, the columns and their
datatypes, the indexes and information about the indexes.

In [ ]:
# Create a TimeSeriesDataFrame
# Use end of period as the time index
# Store and brand combinations label the grain 
# i.e. there is one time series for each unique pair of store and grain
whole_tsdf = TimeSeriesDataFrame(whole_df, 
                                 grain_colnames = ['store', 'brand'],
                                 time_colname = 'WeekLastDay', 
                                 ts_value_colname = 'quantity',
                                 group_colnames = 'store')

# TimeSeriesDataFrame info
print(whole_tsdf.info())

# Find info about the shape of the TimeSeriesDataframe
print(whole_tsdf.shape)

# Describe TimeSeriesDataframe
print(whole_tsdf.describe())

# Index names
print(whole_tsdf.index.names)

# Vector of unique values for specific index column
print(whole_tsdf.index.get_level_values('brand').unique())

# Print sample rows in the data frame from the top.
# Includes the default grain and time axis columns) other than
# the ones explicitly specified.
whole_tsdf[['quantity', 'price']].head()

# The ```TimeSeriesDataFrame.ts_report()``` function generates a comprehensive
# report of the time series data frame, including both general data 
# description and statistics specific to time series data. 
%matplotlib inline
whole_tsdf.ts_report()

## Changing shape of TimeSeriesDataFrame

There are several ways to add or remove columns and rows in a `TimeSeriesDataFrame`. 
In the section below, we will look at some of the most common ways to do so.

In [ ]:
# Add a new column with default value 'NewColumn'
whole_tsdf['defaultcol'] = 'Default'

# Add a new composed column using values from other columns
# "Purchase impact" - ratio of income to per unit cost of item
whole_tsdf['purchimpact'] = whole_tsdf['INCOME']/whole_tsdf['price']

# Add a new column with conditional logic
whole_tsdf['isprofitable'] = np.where(whole_tsdf['price']>=2.0, True, False)

# Add a new column using Python lambda functions
def calc_tax(price):
    return price * 0.1
whole_tsdf['taxamount'] = whole_tsdf.price.apply(calc_tax)

# Remove columns by using Drop
whole_tsdf = whole_tsdf.drop(['defaultcol', 'isprofitable'], axis=1)
whole_tsdf.tail()

# Remove columns by selecting a subset of columns
whole_tsdf_subset = whole_tsdf[['price', 'quantity']]
whole_tsdf_subset.tail()

## Slicers with TimeSeriesDataFrame

Pandas has powerful capabilities to slice dataframe. `TimeSeriesDataFrame` fully supports all of these operations
that are allowed on the native dataframes. Pandas offer the powerful `loc` API to slice objects using labels, but
because `TimeSeriesDataFrame` is multi indexed, the `IndexSlice` or the `slice` APIs can be used in conjunction to perform all sorts of data mangling.

**NOTE:** 
* Since there are multiple indexes in TimeSeriesDataFrame, any slice operation must specify the slice criterion for each index. 
* The indexes are ordinal with the `time_colname` being first and the `grain_colnames` columns in the order they were passed in during the creation.
* The ":" called the null slicer indicates no slicing on the specified index. Not specifying any index simply defaults to a null slice for that ordinal index.

In [ ]:
# Dataframe must be sorted before slice
whole_tsdf.sort_index(inplace=True)

# Slice on a range of values with tuples. Our current TimeSeriesDataFrame
# object has the 'WeekLastDay' time column and two grain columns - 'store' and
# 'brand'. The slicing operations will this specify slice criterion for each of these 
# index columns in that order.
whole_tsdf_singleweek_store5 = whole_tsdf.loc[('1990-06-20 23:59:59', 5,), :]
whole_tsdf_singleweek_store5

In [ ]:
# Get sales of tropicana in store 5 during summer 1990 
whole_tsdf_tropicana_store5_summer90 = whole_tsdf.loc[pd.IndexSlice['1990-06':'1990-09', 5, 'tropicana'], :]
whole_tsdf_tropicana_store5_summer90

In [ ]:
# Get sales of store 5 during summer 1990 
whole_tsdf_store5_summer90 = whole_tsdf.loc[pd.IndexSlice['1990-06':'1990-09', 5], :]
whole_tsdf_store5_summer90

In [ ]:
# Get sales during summer 1990 across all stores
whole_tsdf_summer90 = whole_tsdf.loc[pd.IndexSlice['1990-06':'1990-09',], :]
whole_tsdf_summer90

In [ ]:
# Find the sales for all time periods for a specific brand at a specific store.
# Note the use of axis=0 indicating that the slicing is row level.
# In this case we want rows across all date index for 5 for 'minute.maid' brand.
whole_tsdf_mm = whole_tsdf.loc(axis=0)[:, 5, ['minute.maid']]
whole_tsdf_mm

## Aggregation with TimeSeriesDataFrame

`TimeSeriesDataFrame` supports several aggregation facilities including Pandas' `groupby` API. 
In addition, there are several APIs to slice by well-known index metadata fields that `TimeSeriesDataFrame`
offers to stratify time series data. We will show how some of these can be applied in this section below.

In [ ]:
# Group TimeSeriesDataFrame by any available columns
# Uses native Pandas' groupby
tsdf_groupby = whole_tsdf.groupby(['store', 'brand'])
tsdf_groupby.describe()

In [ ]:
# Add revenue column to TimeSeriesDataFrame.
whole_tsdf['revenue'] = whole_tsdf.price * whole_tsdf.quantity

# Compute total sales in store across entire date range
whole_tsdf.groupby_grain()[['revenue']].sum()

In [ ]:
# Total sales in a store across all available time ranges
# The 'store' is our time-series group column as indicated by the 'group_colnames'.
# We can thus slice the data frame using the groupby_group API in AMLPF.
whole_tsdf_groupby_group = whole_tsdf.groupby_group()[['revenue']]
whole_tsdf_groupby_group.describe()

In [ ]:
# Group TimeSeriesDataFrame by grain columns - 'store' and 'brand'
# Uses AMPPF's `groupby_grain` API.
tsdf_groupby_grain = whole_tsdf.groupby_grain()
tsdf_groupby_grain.describe()

In [ ]:
#
# Computing total sales in a store in a given month of a year
#
import calendar    
year = '1990'
month = '08'
store_val = 2
new_colname = 'yearmonth'  
year_month = str(year) + str(month)
# Clone TimeSeriesDataFrame
df_tmp = whole_tsdf.copy()

# Create a new column and assign the Date index column value
df_tmp = df_tmp.assign(yearmonth=lambda x: \
                       df_tmp.index.get_level_values(0))

# Re-format the datetime value to yearmonth
df_tmp[new_colname] = df_tmp[new_colname].apply(lambda z: z.strftime('%Y%m'))

# Group by the store and year-month and compute sum of TotalSales
# The slice the dataframe for the specific store and yearmonth value
df_tmp = df_tmp.groupby(['store',new_colname]).agg({'revenue': np.sum})\
                .loc[pd.IndexSlice[store_val, year_month], :]
print('Total revenue for store {} in {} {} is {}'\
      .format(store_val, calendar.month_abbr[int('08')], year, df_tmp))

## Merge operations with TimeSeriesDataFrame

`TimeSeriesDataFrame` offers good support for merging and concatenating dataframes. In the section below, we 
show how weather information from the [NOAA's GSOD dataset](https://www7.ncdc.noaa.gov/CDO/GSOD_DESC.txt) can be merged with our `TimeSeriesDataFrame`.

In [ ]:
# Load weather data
weather_1990 = get_a_year_of_daily_weather_data(year=1990)
weather_1991 = get_a_year_of_daily_weather_data(year=1991)
weather_1992 = get_a_year_of_daily_weather_data(year=1992)

# Preprocess weather data
weather_all = pd.concat([weather_1990, weather_1991, weather_1992])
weather_all.reset_index(inplace=True)

# Only use a subset of columns
weather_all = weather_all[['TEMP', 'DEWP', 'WDSP', 'PRCP']]

# Compute the WeekLastDay column, in order to merge with sales data
weekZeroStart = pd.to_datetime('1989-09-07 00:00:00')
weekZeroEnd = pd.to_datetime('1989-09-13 23:59:59')
weather_all['WeekLastDay'] = pd.Series(
    (weather_all.index.get_level_values('YEARMODA') - weekZeroStart), 
    index=weather_all.index.get_level_values('YEARMODA')).apply(lambda n: weekZeroEnd 
                                                                + timedelta(weeks=math.floor(n.days/7)))
weather_all = weather_all.groupby('WeekLastDay').mean()
weather_all = TimeSeriesDataFrame(weather_all, time_colname='WeekLastDay')

# Merge weather data with sales data
whole_tsdf = whole_tsdf.merge(weather_all, how='left', on='WeekLastDay')
whole_tsdf.head()